In [2]:
%run CNNtf.ipynb
%run config.ipynb
%run board_object.ipynb
import sys 
from collections import OrderedDict
import os

In [3]:
#configs: learning rate, batch size, batch size2, epochs

In [ ]:
tf.reset_default_graph()

In [5]:
def policy_move(board_obj_obj, network1, params1_v):
    turn = board_obj_obj.turn
    

    
    
    if turn == 1:

        #argmax needs to be correctly implemented
        #i = prop_choice(loop_model_scores([reformat2(board_obj_obj)], True,  network1)[0])
        i = np.argmax(loop_model_scores([reformat2(board_obj_obj)], True,  network1, params1_v), axis = 1)
        print(i)
        board_obj_obj.move(i)
    elif turn == 2:
        
        #i = prop_choice(loop_model_scores([switch_reformat(reformat2(board_obj_obj))], True,  network1)[0])
        i = np.argmax(loop_model_scores([switch_reformat(reformat2(board_obj_obj))], True,  network1, params1_v), axis = 1)
        print(i)
        board_obj_obj.move(i)
    return (board_obj_obj, i)

In [6]:
def sample_trajectory(network1, params1_v):
    
    board_obj_obj = board_obj()
    states = [board_obj_obj]
    target = []
    board_obj_obj = board_obj_obj.copy()
    while evaluate(board_obj_obj.board) not in [0, 0.5, 1]:
        p = policy_move(board_obj_obj, network1, params1_v)
        (board_obj_obj, i) = (p[0].copy(), p[1]) 
        states.append(board_obj_obj)
        target.append(i)
    outcome = evaluate(board_obj_obj.board)
    
    
    return [states, outcome, target]
    
    

In [7]:
def iteration(params1_v, batch_size, network1):
    #print(params1_v)
    params1_va = params1_v.copy()
    for count in range(proper_batch_size):
        #print(count)
        [states, outcome, target] = sample_trajectory(network1, params1_va)


        


        for i in range(len(target)):
            turn_cur = states[i].turn
            #make sure the cnn_action() network in cnn_action_grad function definition body is IMMUTABLE
            if turn_cur == 1:
                grad = loop_model_grad([reformat2(states[i])], target[i], True, network1, params1_va)
                #grad = np.array(grad)
            elif turn_cur == 2:
                grad = loop_model_grad([switch_reformat(reformat2(states[i]))], target[i], True, network1, params1_va)
                #grad = np.array(grad)
            else:
                print('Turn Error')
                sys.exit()

            if turn_cur == 1:
                #for param, grad_ind in zip(params1, grad):
                    #updates1[param] = param + learning_rate * grad_ind * outcome
                grad = np.array(grad)
                params1_va = np.array(params1_va)
                params1_va = params1_va - learning_rate * grad * outcome    
            elif turn_cur == 2:
                #for param, grad_ind in zip(params2, grad):
                    #updates2[param] = param + learning_rate * grad_ind * outcome
                grad = np.array(grad)
                params1_va = np.array(params1_va)
                params1_va = params1_va - learning_rate * grad * (1 - outcome)     
            else:
                print('Turn Error')
                sys.exit()
                
               
        print('one_loop_done')
    print(params1_va[0][0][0])
    print('break')
    print(params1_v[0][0][0])
    return params1_va


def iteration_action(params1_v, params2_v, batch_size2, network1, network2):
    
    updates1 = OrderedDict()
    updates2 = OrderedDict()
    
    for count in range(proper_batch_size):
        
        [states, outcome, target] = sample_trajectory(network1, network2, params1, params2)
        
        
        T = len(target)
        
        choice = random.choose(list(range(T)))
        choice_2 = random.choose(list(range(T)))
        
        turn_cur = states[choice].turn
        if turn_cur == 1:
            grad = loop_model_score_grad([reformat2(states[choice])], target[choice], True, network1)
            value = loop_model_scores([reformat2(states[choice])], target[choice], True, network1)
        
        elif turn_cur == 2:
            grad = loop_model_score_grad([reformat2(states[choice])], target[choice], True, network2)
            value = loop_model_scores([reformat2(states[choice])], target[choice], True, network2)
        
        
        if turn_cur == 1:
            for param in params1_v:
                updates1[param] = param + learning_rate * grad * (outcome - value)
        
        if turn_cur == 2:
            for param in params2_v:
                updates2[param] = param + learning_rate * grad * (1 - outcome - value)
        print('one_loop_done')  
        
        
                
    return [params1_v, params2_v]

      
    
    

In [ ]:
def REINFORCEloop(epochs):
    network1 = RESIDUAL_net_action_1
    pipe = test_RESIDUAL_net_action_1([1])
    for i in range(epochs):
        #print(i)
        params1 = loop_model_params_get(True, network1)
        u = iteration(params1, batch_size, network1).copy()
        loop_model_params_set(True, network1, u)
        
        
    return network1


def Valueloop(epochs):
    network1 = RESIDUAL_net_value_1
    network2 = RESIDUAL_net_value_2
    for i in range(epochs):
        params1 = lasagne.layers.get_all_params(network1, trainable=True).copy()
        params2 = lasagne.layers.get_all_params(network2, trainable=True).copy()
        [u1, u2] = iteration_action(params1, params2, batch_size, network1, network2).copy()
        lasagne.layers.set_all_param_values(network1, u1) 
        lasagne.layers.set_all_param_values(network2, u2) 
    return [network1, network2]

